# 예측: 회원탈퇴 예측


1. 데이터 읽고 이용데이터 수정
2. 탈퇴전월의 탈퇴고객데이터 작성
3. 지속회원의 데이터 작성
4. 예측할 달의 재적기간 작성
5. 결측치 제거
6. 문자열 변수를 처리할 수 있도록 가공

* 코랩에서 구글 드라이브 파일 연동하기:

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# customer = pd.read_csv('/content/drive/MyDrive/제목 없는 폴더/customer_join.csv')
# uselog = pd.read_csv('/content/drive/MyDrive/제목 없는 폴더/use_log_months.csv')


#### 1. 이번 달과 저번 달의 이용 횟수 작성

In [3]:
import pandas as pd

In [4]:
customer = pd.read_csv('./data/customer_join.csv')
uselog_months = pd.read_csv('./data/use_log_months.csv')


In [5]:
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,야간,6000,일반,3.916667,4.0,6,1,1,2019-04-30,47


In [6]:
uselog_months
# customer.groupby([['customer_id','']])

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7
...,...,...,...
36837,201903,TS995853,8
36838,201903,TS998593,8
36839,201903,TS999079,3
36840,201903,TS999231,6


In [7]:
year_month = list(uselog_months['연월'].unique())
year_month

[201804,
 201805,
 201806,
 201807,
 201808,
 201809,
 201810,
 201811,
 201812,
 201901,
 201902,
 201903]

In [8]:


for i in range(len(year_month)):
  tmp = uselog_months.loc[uselog_months['연월']==year_month[i]]
  tmp.rename(columns={'count':'count_0'},inplace=True)
  tmp_before = uselog_months.loc[uselog_months['연월']==year_month[i-1]]
  tmp_before.rename(columns={'count':'count_1'},inplace=True)
  del tmp_before['연월']
  tmp = tmp.merge(tmp_before, how='left',on='customer_id')
  uselog = pd.DataFrame()
  uselog = pd.concat([uselog,tmp],ignore_index=True)

uselog


/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_1154/3344438854.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.rename(columns={'count':'count_0'},inplace=True)
/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_1154/3344438854.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_before.rename(columns={'count':'count_1'},inplace=True)
/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_1154/3344438854.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

,연월,customer_id,count_0,count_1
0,201903,AS002855,4,5.0
1,201903,AS008805,1,6.0
2,201903,AS009373,5,6.0
3,201903,AS015233,8,4.0
4,201903,AS015315,4,5.0
...,...,...,...,...
2948,201903,TS995853,8,11.0
2949,201903,TS998593,8,7.0
2950,201903,TS999079,3,2.0
2951,201903,TS999231,6,6.0


2.탈퇴회원의 고객 데이터 추가

In [9]:
from dateutil.relativedelta import relativedelta
import datetime


quit_customers = customer.loc[customer['is_deleted'] == 1]
quit_customers['end_date'] = pd.to_datetime(quit_customers['end_date'])
quit_customers['quit_date'] = None

for i in range(len(quit_customers)):
    quit_customers["quit_date"].iloc[i] = quit_customers["end_date"].iloc[i] - relativedelta(months=1)

# quit_customers['quit_date'] = quit_customers['end_date'] - datetime.timedelta(days=36)
quit_customers['연월'] = pd.to_datetime(quit_customers['quit_date']).dt.strftime('%Y%m')


uselog['연월'] = uselog['연월'].astype('str')


quit_uselog = pd.merge(uselog, quit_customers, on=['customer_id','연월'], how='left')
quit_uselog

/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_1154/1523686008.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quit_customers['end_date'] = pd.to_datetime(quit_customers['end_date'])
/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_1154/1523686008.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quit_customers['quit_date'] = None
/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_1154/1523686008.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,quit_date
0,201903,AS002855,4,5.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201903,AS008805,1,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201903,AS009373,5,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201903,AS015233,8,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201903,AS015315,4,5.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2948,201903,TS995853,8,11.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2949,201903,TS998593,8,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2950,201903,TS999079,3,2.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2951,201903,TS999231,6,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
quit_uselog = quit_uselog.dropna(subset=['name'])
quit_uselog.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   연월                 0 non-null      object        
 1   customer_id        0 non-null      object        
 2   count_0            0 non-null      int64         
 3   count_1            0 non-null      float64       
 4   name               0 non-null      object        
 5   class              0 non-null      object        
 6   gender             0 non-null      object        
 7   start_date         0 non-null      object        
 8   end_date           0 non-null      datetime64[ns]
 9   campaign_id        0 non-null      object        
 10  is_deleted         0 non-null      float64       
 11  class_name         0 non-null      object        
 12  price              0 non-null      float64       
 13  campaign_name      0 non-null      object        
 14  mean               0 non-nu

In [327]:
quit_customers

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,quit_date,연월
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,종일,10500,일반,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30 00:00:00,201803
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,주간,7500,일반,3.0,3.0,3,3,0,2018-04-30,23,2018-03-30 00:00:00,201803
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,종일,10500,일반,1.0,1.0,1,1,0,2018-04-30,23,2018-03-30 00:00:00,201803
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,야간,6000,일반,3.5,3.5,4,3,1,2018-05-31,23,2018-04-30 00:00:00,201804
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,종일,10500,일반,3.0,3.0,3,3,0,2018-05-31,23,2018-04-30 00:00:00,201804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4049,OA143589,XXXXX,C03,F,2019-01-15,2019-02-28,CA1,1,야간,6000,일반,4.5,4.5,6,3,0,2019-02-28,1,2019-01-28 00:00:00,201901
4050,GD367382,XXXX,C01,M,2019-01-15,2019-03-31,CA1,1,종일,10500,일반,7.0,6.0,12,3,0,2019-03-31,2,2019-02-28 00:00:00,201902
4068,OA034442,XXXXX,C02,M,2019-02-05,2019-03-31,CA1,1,주간,7500,일반,7.0,7.0,11,3,0,2019-03-31,1,2019-02-28 00:00:00,201902
4086,PL508278,XXXXX,C03,M,2019-02-09,2019-03-31,CA1,1,야간,6000,일반,7.0,7.0,10,4,0,2019-03-31,1,2019-02-28 00:00:00,201902


#### 지속 회원 데이터 작성

In [11]:
from dateutil.relativedelta import relativedelta
import datetime


stay_customers = customer.loc[customer['is_deleted'] == 0]

stay_uselog = uselog_months.merge(stay_customers, on='customer_id',how='left')




In [12]:
print(len(stay_uselog))
stay_uselog = stay_uselog.dropna(subset=['name'])
print(len(stay_uselog))

36842
29576


In [13]:
# undersampling

stay_uselog = stay_uselog.sample(frac=1).reset_index(drop=True)
stay_uselog = stay_uselog.drop_duplicates(subset=['customer_id'])


In [14]:
predict_data = pd.concat([quit_uselog,stay_uselog],ignore_index=True)
print(len(predict_data))



2842


#### 4. 재적기간 작성

In [332]:
predict_data

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,quit_date,count
0,201811,OA076062,NaN,NaN,XXXXX,C01,M,2017-10-01,NaN,CA1,...,일반,6.583333,7.0,10.0,4.0,1.0,2019-04-30,18.0,NaN,4.0
1,201812,PL405819,NaN,NaN,XXXX,C01,M,2018-12-15,NaN,CA3,...,입회비무료,8.500000,8.5,9.0,8.0,1.0,2019-04-30,4.0,NaN,8.0
2,201805,TS132003,NaN,NaN,XX,C02,F,2017-11-01,NaN,CA1,...,일반,6.083333,6.5,9.0,1.0,1.0,2019-04-30,17.0,NaN,9.0
3,201902,GD900956,NaN,NaN,XXX,C01,M,2015-12-01,NaN,CA1,...,일반,5.083333,5.0,8.0,2.0,1.0,2019-04-30,40.0,NaN,4.0
4,201808,AS637476,NaN,NaN,XX,C02,F,2015-07-01,NaN,CA1,...,일반,4.250000,4.5,5.0,3.0,1.0,2019-04-30,45.0,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2837,201903,AS921334,NaN,NaN,XXXXX,C02,F,2019-03-04,NaN,CA1,...,일반,9.000000,9.0,9.0,9.0,1.0,2019-04-30,1.0,NaN,9.0
2838,201903,HD126303,NaN,NaN,XXX,C03,F,2019-03-11,NaN,CA1,...,일반,9.000000,9.0,9.0,9.0,0.0,2019-04-30,1.0,NaN,9.0
2839,201903,AS076940,NaN,NaN,XXXXX,C01,F,2019-03-12,NaN,CA1,...,일반,7.000000,7.0,7.0,7.0,0.0,2019-04-30,1.0,NaN,7.0
2840,201903,GD748823,NaN,NaN,XXXX,C02,F,2019-03-04,NaN,CA1,...,일반,6.000000,6.0,6.0,6.0,1.0,2019-04-30,1.0,NaN,6.0


In [18]:
predict_data['period'] = 0
predict_data['now_date'] = pd.to_datetime(predict_data['연월'], format='%Y%m')
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])

predict_data.info()






<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2842 entries, 0 to 2841
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   연월                 2842 non-null   datetime64[ns]
 1   customer_id        2842 non-null   object        
 2   count_0            0 non-null      float64       
 3   count_1            0 non-null      float64       
 4   name               2842 non-null   object        
 5   class              2842 non-null   object        
 6   gender             2842 non-null   object        
 7   start_date         2842 non-null   datetime64[ns]
 8   end_date           0 non-null      object        
 9   campaign_id        2842 non-null   object        
 10  is_deleted         2842 non-null   float64       
 11  class_name         2842 non-null   object        
 12  price              2842 non-null   float64       
 13  campaign_name      2842 non-null   object        
 14  mean    

In [20]:
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)



predict_data.head()


/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_1154/577908157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data["period"][i] = int(delta.years*12 + delta.months)
/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_1154/577908157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data["period"][i] = int(delta.years*12 + delta.months)
/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_1154/577908157.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,median,max,min,routine_flg,calc_date,membership_period,quit_date,count,period,now_date
0,1970-01-01 00:00:00.000201804,PL758737,NaN,NaN,XX,C01,M,2017-07-01,NaN,CA2,...,6.5,8.0,2.0,1.0,2019-04-30,21.0,NaN,6.0,-569,1970-01-01 00:00:00.000201804
1,1970-01-01 00:00:00.000201902,HI247770,NaN,NaN,XXXXX,C01,F,2017-12-01,NaN,CA3,...,7.0,10.0,4.0,1.0,2019-04-30,16.0,NaN,5.0,-574,1970-01-01 00:00:00.000201902
2,1970-01-01 00:00:00.000201805,GD323923,NaN,NaN,XX,C01,M,2015-10-01,NaN,CA1,...,5.0,7.0,2.0,1.0,2019-04-30,42.0,NaN,3.0,-548,1970-01-01 00:00:00.000201805
3,1970-01-01 00:00:00.000201804,TS999855,NaN,NaN,XXXXXX,C01,F,2017-04-01,NaN,CA1,...,4.0,7.0,3.0,1.0,2019-04-30,24.0,NaN,6.0,-566,1970-01-01 00:00:00.000201804
4,1970-01-01 00:00:00.000201806,IK572153,NaN,NaN,XXXXX,C01,F,2016-07-01,NaN,CA2,...,5.5,8.0,2.0,1.0,2019-04-30,33.0,NaN,6.0,-557,1970-01-01 00:00:00.000201806


결측치 제거

In [21]:
predict_data.isnull().sum()
predict_data = predict_data.dropna(subset=['count_1'])    # subset= : 해당 칼럼에 결측치가 있을 때 삭제, 그 외의 칼럼에 결측치가 있어도 삭제 X
predict_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   연월                 0 non-null      datetime64[ns]
 1   customer_id        0 non-null      object        
 2   count_0            0 non-null      float64       
 3   count_1            0 non-null      float64       
 4   name               0 non-null      object        
 5   class              0 non-null      object        
 6   gender             0 non-null      object        
 7   start_date         0 non-null      datetime64[ns]
 8   end_date           0 non-null      object        
 9   campaign_id        0 non-null      object        
 10  is_deleted         0 non-null      float64       
 11  class_name         0 non-null      object        
 12  price              0 non-null      float64       
 13  campaign_name      0 non-null      object        
 14  mean               0 non-nu

#### 문자형 범주형 처리

In [ ]:
target_cols = ['campaign_name','class_name','gender','count_1','routine_flg','membership_period','is_deleted']
predict_cols = final_uselog[target_cols]
predict_cols.head()









In [ ]:
predict_cols = pd.get_dummies(predict_cols)
predict_cols.head()

In [ ]:
del predict_cols['campaign_name_일반']
del predict_cols['class_name_야간']
del predict_cols['gender_M']
predict_cols.head()

decision tree model 작성

In [ ]:
predict_cols.dropna(axis=0,inplace=True)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

# 탈퇴 데이터와 유지 데이터 개수 정리 -> 유지 데이터 임의로 1204건 추출해서 50:50으로 비율 맞출 예정

exit = predict_cols.loc[predict_cols['is_deleted'] == 1]
stay = predict_cols.loc[predict_cols['is_deleted'] == 0].sample(len(exit))


print(len(exit),len(stay))





In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = predict_cols.drop('is_deleted',axis=1,inplace=False)
y = predict_cols['is_deleted']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=11)

dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train,y_train)

pred = dt_clf.predict(X_test)
scores = accuracy_score(y_test,pred)

print(pred)








In [ ]:
result_test = pd.DataFrame({'y_test':y_test,'y_pred':pred})
result_test.sample(10)








8. 예측 모델 평가하고 모델 튜닝

In [ ]:
correct = len(result_test.loc[result_test['y_test'] == result_test['y_pred']])
data_count = len(result_test)
score_test = correct / data_count
print(score_test)











하이퍼파라미터 튜닝

In [ ]:
X = predict_cols.drop('is_deleted',axis=1,inplace=False)
y = predict_cols['is_deleted']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=11)

dt_clf = DecisionTreeClassifier(random_state=0,max_depth=5)
dt_clf.fit(X_train,y_train)

print(dt_clf.score(X_test,y_test))
print(dt_clf.score(X_train,y_train))








In [ ]:
importance = pd.DataFrame({'feature_names': X.columns, 'coefficient':dt_clf.feature_importances_})

importance



10. 회원탈퇴예측

In [ ]:
count_1 = 3
routine_flg = 1
membership_period = 10
campaign_name = '입회비무료'
class_name = '종일'
gender = 'M'

In [ ]:
if campaign_name == '입회비반값할인':
  campaign_name_list = [1,0]
elif campaign_name == '입회비무료':
  campaign_name_list = [0,1]
elif campaign_name == '무료':
  campaign_name_list = [0,0]
if class_name == '종일':
  class_name_list = [1,0]
elif class_name == '주간':
  class_name_list = [0,1]
elif class_name == '야간':
  class_name_list = [0,0]
if gender == 'M':
  gender_list = [1]
elif gender == 'F':
  gender_list = [0]

input_data = [count_1, routine_flg, membership_period]
input_data.extend(list(campaign_name_list))
input_data.extend(class_name_list)
input_data







In [ ]:
print(dt_clf.predict([input_data]))
print(dt_clf.predict_proba([input_data]))









